## Scraping Restaurant Reviews from RestaurantGuru

#### Purpose
This notebook is the **core data collection component** of the project.
It scrapes customer reviews for each restaurant previously identified in the link scraping step.

---

#### What This Notebook Does
- Loads restaurant URLs generated by the link scraping notebook
- Visits each restaurant profile page
- Extracts review titles and review texts
- Saves the scraped reviews into multiple CSV files

---

#### Role in the Project
This notebook transforms raw restaurant links into **structured review data**,
which forms the main dataset for analysis or downstream applications.

---

#### Output
- Multiple CSV files containing restaurant reviews
- Files are later consolidated using:
  `merge_restaurant_reviews.ipynb`

---

#### Notes
- Randomized waiting times are used to mimic human browsing behavior
- The process can be time-consuming for large cities
- Failed pages are skipped to keep the scraping process robust




In [ ]:
import pickle
import os
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import random
from random import uniform
import pandas as pd
import glob


Preparations

In [ ]:
#-----Onedrive BENUTZERNAME (z.B. ElianeTuchborn)
username="ElianeTuchborn"

#-----Stadt des Link-Datensatzes
stadt = "Frankfurt"

#-----Hier Index für ersten Restaurant-Link der gescraped werden soll festlegen
index_start = 415

#-----Hier Index für letzten Restaurant-Link der gescraped werden soll festlegen
index_stop = 4798



Start Webscraping

In [ ]:
# Dateipfad und Dateiname der CSV-Datei
dateipfad_input = r"C:\Users\\" + username + r"\Outputs_Links\\"
dateiname_input = "01Links_" + stadt + ".csv"

dateipfad_output = r"C:\Users\\" + username + r"\Outputs_Bewertungen\\"
dateiname_output = "Bewertungen_" + stadt + ".csv"


# Definiere den Pfad zum Chrome Webdriver
options = Options() #erstelle Options-Objekt um m verschiedene Konfigurationsoptionen für den Webdriver anzupassen
PATH = "C:\\Users\\" + username + "\\chromedriver.exe"
driver = webdriver.Chrome(PATH, options=options) #definiere Driver


# CSV-Datei importieren und als DataFrame speichern
df_links = pd.read_csv(dateipfad_input + dateiname_input)
list_links = df_links["Link"].tolist()


for i in range(index_start, len(list_links)):
    if i <= index_stop:
        link = list_links[i]
        print(i)
        print(link)
        try:
            # Erstelle eine Instanz der Chrome-Optionen
            options = Options()
            options.add_argument("--disable-blink-features=AutomationControlled")  # Deaktiviert bestimmte Blink-Funktionen im Browser, die von Automatisierungswerkzeugen verwendet werden können.
            options.add_argument("--disable-extensions")  # Deaktiviert Browser-Erweiterungen, um mögliche Konflikte oder unerwünschte Verhaltensweisen zu vermeiden.
            options.add_argument("--disable-infobars")  # Deaktiviert Infobalken im Browser, die Informationen anzeigen können.
            options.add_argument("--disable-notifications")  # Deaktiviert Browser-Benachrichtigungen, um Unterbrechungen während des Automatisierungsprozesses zu vermeiden.
            options.add_argument("--disable-popup-blocking")  # Deaktiviert das Blockieren von Popup-Fenstern im Browser.
            options.add_argument("--disable-web-security")  # Deaktiviert die Web-Sicherheitseinstellungen im Browser, um den Zugriff auf Ressourcen von verschiedenen Domains zu ermöglichen.
            options.add_argument("--disable-dev-shm-usage")  # Deaktiviert die gemeinsame Nutzung des /dev/shm-Speichers im Browser, was die Speichernutzung reduzieren kann.
            options.add_argument("--disable-gpu")  # Deaktiviert die Hardwarebeschleunigung im Browser.
            options.add_argument("--window-size=1200,800")  # Setzt die Größe des Browserfensters auf 1200 Pixel Breite und 800 Pixel Höhe.
            options.add_argument("--disable-features=VizDisplayCompositor")  # Deaktiviert das Anzeigen von Inhalten mit dem VizDisplayCompositor.
            options.add_argument("--window-size=1366,768")  # Setzt die Größe des Browserfensters auf 1366 Pixel Breite und 768 Pixel Höhe.

            # headless Browser
            #options.add_argument("--headless")

            # Warte
            time.sleep(uniform(1, 3))

            # Öffne die Website
            driver.get(link)

            # Warte
            time.sleep(uniform(1, 3))

            # Scrolle bis ans Ende der Seite
            current_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(uniform(1, 2))
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == current_height:
                    break
                current_height = new_height

            # Warte
            time.sleep(uniform(3, 4))

            
            # Finde alle Elemente mit Kommentaren, Bewertungen, Benutzernamen, ID und Datum
            container = driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[1]/div/div[4]") # Definiere Container; X-Path gibt spezifische Funktion des div-Elements an

            comments = container.find_elements_by_class_name("text_full")
            elements = container.find_elements_by_class_name("user_info__head")
            scores = container.find_elements_by_css_selector("[data-score]")  # Finde Elemente mit "data-score" Attribut
            id_elements = container.find_elements_by_css_selector("[data-id]")  # Finde Elemente mit "data-id" Attribut

            # Extrahierte Kommentare in Datensatz
            dfcomment = pd.DataFrame([comment.text for comment in comments], columns=["Kommentar"])

            # Extrahierte Elemente in Datensatz und splitte nach Benutzer, Datum und Plattform
            element_list = [element.text for element in elements]
            dfelement = pd.DataFrame(element_list, columns=["Element"])
            dfelement[["Name", "Datum", "Plattform"]] = dfelement["Element"].str.split(" vor | auf ", expand=True)

            # Extrahiere den Wert des "data-score" Attributs als Text
            dfscores = pd.DataFrame([score.get_attribute("data-score") for score in scores], columns=["Score"])

            # Extrahiere den Wert des "data-id" Attributs als Text
            dfid = pd.DataFrame([id_element.get_attribute("data-id") for id_element in id_elements], columns=["ID"])

            # Datensätze horizontal aneinanderhängen
            df_bewertungen = pd.concat([dfcomment, dfelement, dfscores, dfid], axis=1)

        

            #Zeitangeben cleanen
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("1 Monate", "04.23")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("2 Monate", "03.23")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("3 Monate", "02.23")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("4 Monate", "01.23")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("5 Monate", "12.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("6 Monate", "11.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("7 Monate", "10.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("8 Monate", "09.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("9 Monate", "08.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("10 Monate", "07.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("11 Monate", "06.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("12 Monate", "05.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("ein Jahr", "00.22")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("2 Jahre", "00.21")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("3 Jahre", "00.20")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("4 Jahre", "00.19")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("5 Jahre", "00.18")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("6 Jahre", "00.17")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("7 Jahre", "00.16")
            df_bewertungen["Datum"]=df_bewertungen["Datum"].replace("8 Jahre", "00.15")


            # Speichern
            filename = f"01Bewertungen_{stadt}{i}.csv"
            df_bewertungen.to_csv(os.path.join(dateipfad_output, filename), index=False)

            # Zeige Fortschritt an
            print("Restaurant", i , "von", len(list_links), "scrape done.")

            # Schließe Browser
            #driver.quit()
            time.sleep(uniform(15, 24))

        except Exception as e:
            print("Fehler:", e)


            # Speichere bisher gesammelte Daten als CSV
            #filename = f"Bewertungen_{stadt}_{index_start}_{i-1}.csv"
            #df_bewertungen.to_csv(os.path.join(dateipfad_output, filename), index=False)

    
            break
    else: print("Alle Restaurants wurden gescraped")    

